In [22]:
import numpy as np
import pandas as pd

In [23]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def derivative(x):
    return sigmoid(x) * (1-sigmoid(x))

class NeuralNetwork:
    def __init__(self, layers, learning_rate, weights = None, biases = None):
        '''layers : list representing number of neurons in each layer'''
        self.layers = layers
        if(weights is None):
            self.weights = self.initialize_weights(layers)
        else:
            self.weights=weights
        if(biases is None):
            self.biases = self.initialize_biases(layers)
        else:
            self.biases=biases
        self.learning_rate = learning_rate
        self.z =[]
        self.errors=[]


    def initialize_weights(self, layers):
        matrices =[]
        for i in range(len(layers)-1):
            matrix = np.random.uniform(0,1,(layers[i], layers[i+1]))
            matrices.append(matrix)
        return matrices
    def initialize_biases(self, layers):
        biases=[]
        for i in range(1,len(layers)):
            bias = np.random.uniform(0,1,(layers[i]))
            biases.append(bias)
        return biases
    
    def forward_propagation(self, input):
        current = input
        value_list = []
        value_list.append(input)
        for i in range(len(self.weights)-1):
            z = np.dot(current, self.weights[i]) + self.biases[i]
            value_list.append(z)
            a = sigmoid(z)
            current = a 
        current = np.dot(current, self.weights[-1]) + self.biases[-1]
        value_list.append(current)
        self.z = value_list
        return current
    
    def backward_propagation(self,y, y_hat):
        
        last_layer_error = (y_hat - y)*derivative(self.z[-1].item())
        self.errors.append(last_layer_error)
        self.biases[-1]-= self.learning_rate*last_layer_error
    
        for i in range(len(self.weights)-2, -1, -1):
            errors=[]
            for j in range(len(self.z[i+1])):
                z = self.z[i+1][j]
                next_layer_error = self.errors[-1]
                weights = self.weights[i+1][j]
                neuron_error= np.dot(weights, next_layer_error) * derivative(z)
                errors.append(neuron_error)
            self.errors.append(errors)
       
        self.errors[1:] = self.errors[1:][::-1]
        self.errors.append(self.errors.pop(0))
        
       #weights change
        for layer_ind in range(len(self.weights)-1, -1, -1):
            
            for i in range(self.weights[layer_ind].shape[0]):
                for j in range(self.weights[layer_ind].shape[1]):
                    z=self.z[layer_ind][i]

                    e= self.errors[layer_ind][j]
                    self.weights[layer_ind][i][j] -= self.learning_rate*sigmoid(z)*e
        
        #bias change
        for layer_ind in range(len(self.biases)):
            layer_errors = self.errors[layer_ind]
            for i in range(len(layer_errors)):
                self.biases[layer_ind][i] -= self.learning_rate*layer_errors[i]

    def train(self, X_train, y_train, target_mse, epochs):
        X_train = np.array(X_train)
        y_train = np.array(y_train)

        for epoch in range(epochs):
            print(f'Epoch : {epoch}')
            total_error =0
            for x,y in zip(X_train, y_train):
                y_hat = self.forward_propagation(x)
                self.backward_propagation(y, y_hat)
                total_error+= (y-y_hat)**2
            mse = np.mean(total_error)
            print(f'MSE: {mse}')
            if(mse<target_mse):
                print(f"Training stopped at epoch {epoch} with MSE {mse:.6f}")
                print(f'Current weights: {self.weights}, Current biases: {self.biases}')
                break
        else:
            print("Reached max epochs without hitting target MSE.")

    def predict(self, X):
        return self.forward_propagation(X)

                    


In [24]:
df_train = pd.read_csv('data/square-simple-training.csv', index_col=0)
from sklearn.preprocessing import MinMaxScaler
X_train = df_train.iloc[:,0].values.reshape(-1,1)
y_train = df_train.iloc[:,1].values.reshape(-1,1)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Dostosowanie do danych i transformacja
y_train_scaled = scaler.fit_transform(y_train)




In [25]:
network = NeuralNetwork([1,2,3,1], 0.1)
network.train(X_train, y_train, 4, 10000)

Epoch : 0
MSE: 962754.9952981081
Epoch : 1
MSE: 956881.4617235423
Epoch : 2
MSE: 956809.112333843
Epoch : 3
MSE: 956738.1398023256
Epoch : 4
MSE: 956668.4941474259
Epoch : 5
MSE: 956600.1280368598
Epoch : 6
MSE: 956532.9966044076
Epoch : 7
MSE: 956467.0572822315
Epoch : 8
MSE: 956402.2696471743
Epoch : 9
MSE: 956338.5952796675
Epoch : 10
MSE: 956275.9976340213
Epoch : 11
MSE: 956214.4419190258
Epoch : 12
MSE: 956153.8949878837
Epoch : 13
MSE: 956094.325236628
Epoch : 14
MSE: 956035.7025102362
Epoch : 15
MSE: 955977.9980157743
Epoch : 16
MSE: 955921.1842419226
Epoch : 17
MSE: 955865.234884358
Epoch : 18
MSE: 955810.1247764622
Epoch : 19
MSE: 955755.8298249333
Epoch : 20
MSE: 955702.3269498651
Epoch : 21
MSE: 955649.5940289507
Epoch : 22
MSE: 955597.6098454566
Epoch : 23
MSE: 955546.3540396777
Epoch : 24
MSE: 955495.8070635963
Epoch : 25
MSE: 955445.950138486
Epoch : 26
MSE: 955396.765215249
Epoch : 27
MSE: 955348.234937265
Epoch : 28
MSE: 955300.3426055704
Epoch : 29
MSE: 955253.0721461

KeyboardInterrupt: 

In [ ]:
df_test = pd.read_csv('data/square-simple-test.csv', index_col=0)

X_test = df_test.iloc[:,0].values.reshape(-1,1)
y_test = df_test.iloc[:,1].values.reshape(-1,1)

X_test_scaled = scaler.transform(X_test)  # Dostosowanie do danych i transformacja

y_pred = network.predict(X_test)
mse = np.mean((y_test-y_pred)**2)
print("MSE",mse)

MSE 8628.205742596683


In [ ]:
y_pred

array([[0.09391273],
       [0.10354093],
       [0.05463142],
       [1.65840772],
       [1.14181151],
       [1.93992006],
       [0.09061907],
       [2.24116236],
       [0.07068272],
       [0.09381929],
       [0.96785391],
       [0.06585772],
       [1.65315427],
       [0.05658484],
       [0.53894825],
       [1.62454659],
       [0.08786323],
       [0.10999336],
       [0.09579575],
       [0.55241555],
       [2.23376989],
       [0.08646703],
       [0.16137776],
       [0.08642089],
       [0.07540281],
       [0.08916447],
       [0.05402611],
       [0.06694863],
       [0.09861151],
       [0.08814084],
       [1.09478546],
       [0.05509536],
       [0.05494456],
       [0.05639445],
       [0.0542196 ],
       [0.09568197],
       [1.97940649],
       [0.14145993],
       [1.78749371],
       [1.55610911],
       [2.2181292 ],
       [2.12102381],
       [0.06696084],
       [0.07630456],
       [1.24951982],
       [1.31131044],
       [2.21796626],
       [1.778